In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import numpy as np
import random
from sklearn.covariance import EllipticEnvelope

class DeepConvKMeans(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, num_clusters, lambda_reg=0.001, mu=1.0):
        super(DeepConvKMeans, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size, padding=kernel_size // 2)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size, padding=kernel_size // 2)
        self.lambda_reg = lambda_reg
        self.mu = mu
        self.num_clusters = num_clusters

    def forward(self, x):
        x = F.selu(F.max_pool3d(self.conv1(x), kernel_size=2))
        z = F.max_pool3d(self.conv2(x), kernel_size=2)
        
        return z

    def kmeans_loss(self, Z, H):
        ZHT = Z @ H
        HHT = H.T @ H
        HHT_inv = torch.inverse(HHT)
        loss = torch.norm(Z - ZHT @ HHT_inv @ H.T, p='fro')**2
        return loss

    def regularization_loss(self, T1, T2):
        reg_loss = 0
        for Ti in [T1, T2]:
            Ti_matrix = Ti.view(Ti.size(0), -1)
            reg_loss += self.lambda_reg * (torch.norm(Ti_matrix, p='fro')**2 - torch.logdet(Ti_matrix @ Ti_matrix.T))
        return reg_loss

    def total_loss(self, Z, H, T1, T2):
        Z_flat = Z.view(Z.size(0), -1)
        loss_kmeans = self.kmeans_loss(Z_flat, H)
        loss_reg = self.regularization_loss(T1, T2)
        total_loss = loss_kmeans + loss_reg
        return total_loss

def train(model, data, num_epochs, num_clusters):
    model = model.cuda()
    data = data.cuda()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        Z = model(data)
        Z_flat = Z.view(Z.size(0), -1)
        H = torch.rand(Z_flat.size(1), num_clusters).to(Z.device)
        T1, T2 = model.conv1.weight, model.conv2.weight
        loss = model.total_loss(Z, H, T1, T2)
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [2]:
data = np.load('./fulldata_3d.npy')  # Load data (131, 50, 50, 50)
data = np.expand_dims(data, axis=1) # Shape: (131, 1, 50, 50, 50)
data.shape

(131, 1, 50, 50, 50)

In [3]:
# def set_seed(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# set_seed(42)

In [4]:
tensor_data = torch.tensor(data, dtype=torch.float32).cuda()
num_clusters = 4

model = DeepConvKMeans(in_channels=1, out_channels=8, kernel_size=3, num_clusters=num_clusters).cuda()

train(model, tensor_data, num_epochs=2000, num_clusters=num_clusters)

# print(f"Cluster assignments: {h}")
# print(f"Cluster centers: {kmeans_center}")

Epoch [1/2000], Loss: 16456.41796875
Epoch [2/2000], Loss: 11388.5556640625
Epoch [3/2000], Loss: 8034.5556640625
Epoch [4/2000], Loss: 6003.97509765625
Epoch [5/2000], Loss: 4834.63232421875
Epoch [6/2000], Loss: 4170.8212890625
Epoch [7/2000], Loss: 3716.95556640625
Epoch [8/2000], Loss: 3373.362548828125
Epoch [9/2000], Loss: 3097.223876953125
Epoch [10/2000], Loss: 2846.94775390625
Epoch [11/2000], Loss: 2649.859375
Epoch [12/2000], Loss: 2488.910888671875
Epoch [13/2000], Loss: 2371.98583984375
Epoch [14/2000], Loss: 2255.981201171875
Epoch [15/2000], Loss: 2151.49365234375
Epoch [16/2000], Loss: 2035.476806640625
Epoch [17/2000], Loss: 1934.9849853515625
Epoch [18/2000], Loss: 1830.1429443359375
Epoch [19/2000], Loss: 1714.7734375
Epoch [20/2000], Loss: 1627.729248046875
Epoch [21/2000], Loss: 1534.0223388671875
Epoch [22/2000], Loss: 1463.35595703125
Epoch [23/2000], Loss: 1400.4122314453125
Epoch [24/2000], Loss: 1344.6318359375
Epoch [25/2000], Loss: 1296.7890625
Epoch [26/200

In [5]:
features = model(tensor_data)
print(features.shape)

torch.Size([131, 8, 12, 12, 12])


In [6]:
from sklearn.decomposition import PCA

Z_flat = features.view(features.size(0), -1).detach().cpu().numpy()

pca = PCA(n_components=3)
features_pca = pca.fit_transform(Z_flat)

In [ ]:

ee = EllipticEnvelope(support_fraction=1, contamination=0.05)
ee.fit(features_pca)
# mahalanobis_distances = ee.mahalanobis(Z_flat)

EllipticEnvelope(support_fraction=1)

In [8]:
eee = ee.predict(features_pca)

In [ ]:
eee

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1])

In [10]:
import numpy as np 
import glob

meshfiles = []
for file in glob.glob("../fulldata-stl/*"):
    meshfiles.append(file)


In [11]:
indices_of_minus_one = np.where(eee == -1)[0]
indices_of_minus_one

array([  9,  10,  18,  24,  26,  27,  28,  35,  58,  92, 106, 126, 129])

In [12]:
pred_outliers = []
for i in indices_of_minus_one:
  pred_outliers.append(meshfiles[i].replace("../fulldata-stl/", ""))
  print(meshfiles[i].replace("../fulldata-stl/", ""))

10246_région1.stl
20148_région1.stl
10195_région1.stl
20215_région1.stl
20183_région1.stl
10182_région2.stl
10143_région4.stl
10134_région4.stl
10159_région4.stl
20189_région2.stl
20187_région1.stl
20206_région1.stl
10262_région1.stl


In [13]:
outliers = [
"20148_région1.stl",
"20183_région1.stl",
"20187_région1.stl",
"10204_région1.stl",
"20151_région4.stl",
"20212_région1.stl",
"10244_région1.stl"
]

In [14]:
common_strings = list(set(pred_outliers) & set(outliers))
common_strings

['20183_région1.stl', '20148_région1.stl', '20187_région1.stl']